In [1]:
import glob, sys

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, O, M, WikidataSemanticModelHelper
sys.path.append(str(ROOT_DIR / "sm_annotator"))

In [2]:
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

In [3]:
cwd = ROOT_DIR / "examples/t2dv2"
tables = [I.W2WTable.from_csv_file(infile) for infile in glob.glob(str(cwd / "tables/*.csv")) if infile.find("table_linker") == -1]

In [4]:
annotations = [grams.annotate(table) for table in tqdm(tables)]

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
from sm_annotator.prelude import Annotator, BatchAnnotator, widgets, GRAMSAnnotatorAssistant, SliderApp

qnodes = grams.qnodes.cache_dict()
wdprops = grams.wdprops
wdclasses = grams.wdclasses.cache_dict()

assistant = GRAMSAnnotatorAssistant([
    dict(table=table, sg=annotation.sg, dg=annotation.dg)
    for annotation, table in zip(annotations, tables)
], qnodes, wdprops)
annotator = Annotator(qnodes, wdclasses, wdprops, cwd / "ground-truth", 
                          eshost='http://mira.isi.edu:9200', username='', password='', dev=False, assistant=assistant)
batch_annotator = BatchAnnotator(annotator)
batch_annotator.render(same_tab=True)

Output()

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='f4bb67be-14b9-4931-9fbe-2313f6fc1a3d')

HTML(value='<div id="e1da0f57-7287-414b-bb5c-06b0bbc57d9f"></div>')

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='37b37e69-898f-46af-8f32-951a73617479')

<IPython.core.display.Javascript object>

In [6]:
batch_annotator.batch_annotate([
    (table.get_friendly_fs_id(), "", table)
    for table in tables
], start_index=0)

In [13]:
!ls "{cwd}/ground-truth"

29414811_2_4773219892816395776.json
